## What is RAG?

RAG stands for Retrieval Augmented Generation.

* **Retrieval** - Seeking relevant information from a source given a query
* **Augmented** - Using the relevant retrieved information to modify an input to a generative model
* **Generation** - Generating an output given an input

 ## What we're going to build

1. Open a PDF document.
2. Format the text of the PDF textbook ready for an embedding model.
3. Embed all of the chunks of text in the textbook and turn them into numerical representation which we can store for later.
4. Build a retrieval system that uses vector search to find relevant chunks of text based on a query.
5. Create a prompt that incorporates the retrieved pieces of text.
6. Generate an answer to a query based on passages from the textbook.

## 1. Document Retrieval

In [1]:
import os
import requests

In [2]:
filename = "human-nutrition-text.pdf"
url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
response = requests.get(url)

if response.status_code == 200:
    with open(filename, "wb") as file:
        file.write(response.content) 
    print(f"The file has been download and saved as {filename}")
else:
    print(f"Failed to download the file")

The file has been download and saved as human-nutrition-text.pdf


In [3]:
!pip install PyMuPDF
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.2 MB/s eta 0:00:00:00:0100:01


## Text Preprocessing

In [10]:
import fitz
from tqdm.auto import tqdm
import pandas as pd
import random

In [5]:
def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number - 41, 
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(", ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

In [7]:
pdf_path = "/kaggle/working/human-nutrition-text.pdf"

In [8]:
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [11]:
random.sample(pages_and_texts, k=2)

[{'page_number': 994,
  'page_char_count': 1596,
  'page_word_count': 265,
  'page_sentence_count_raw': 26,
  'page_token_count': 399.0,
  'text': '• Temperature. Between 40°F and 140°F, which is called the  danger zone, bacteria grow rapidly.  • Time. More than two hours in the danger zone.  • Water. High moisture content is helpful. Fresh fruits and  vegetables have the highest moisture content.  • Oxygen. Most microorganisms need oxygen to grow and  multiply, but a few are anaerobic and do not.  • Acidity and pH Level. Foods that have a low level of acidity (or a  high pH level) provide an ideal environment, since most  microorganisms grow best around pH 7.0 \xa0and not many will  grow below pH 4.0 . Examples of higher pH foods include egg,  meat, seafood, milk, and corn. Examples of low pH foods  include citrus fruits, sauerkraut, tomatoes, and pineapples.  • Nutrient Content. Microorganisms need protein, starch,  sugars, fats, and other compounds to grow. Typically high- protein f

In [12]:
df = pd.DataFrame(pages_and_texts)

In [13]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,11,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,2,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,1,199.25,Contents Preface University of Hawai‘i at Mā...


In [14]:
df.tail()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
1203,1162,1676,252,1,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...
1204,1163,1617,254,6,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...
1205,1164,1715,261,8,428.75,Flashcard Images Note: Most images in the fla...
1206,1165,1733,268,4,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...
1207,1166,257,44,1,64.25,23. Vitamin D reused “The Functions of Vitamin...


In [15]:
df.shape

(1208, 6)

In [16]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,11.05,287.00
std,348.86,560.38,95.76,8.95,140.10
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,5.00,190.50
50%,562.50,1231.50,214.50,10.00,307.88
75%,864.25,1603.50,271.00,15.00,400.88
max,1166.00,2308.00,429.00,106.00,577.00


## Text Chunking

In [35]:
from spacy.lang.en import English
import re

In [18]:
nlp = English()
nlp.add_pipe("sentencizer")

In [19]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [20]:
random.sample(pages_and_texts, k=1)

[{'page_number': 955,
  'page_char_count': 784,
  'page_word_count': 155,
  'page_sentence_count_raw': 3,
  'page_token_count': 196.0,
  'text': 'Image by  Allison  Calabrese /  CC BY 4.0  of glucose is used as well. Glucose differs from fatty acids where  glycogen storages can be depleted. As glycogen stores are depleted,  fatigue will eventually set in.  Figure 16.6 The Effect of Exercise Intensity on Fuel Sources  The Fat-Burning Zone  The fat-burning zone is a low intensity aerobic activity  that keeps your heart rate between 60 and 69% of your  maximum heart rate. The cardio zone on the other hand is  a high intensity aerobic activity that keeps the heart rate  between about 70 to 85% of your maximum heart rate. So  which zone do you burn the most fat in? Technically, your  body burns a higher percentage of of calories from fat  during a low intensity aerobic activity but there’s more to it  Fuel Sources  |  955',
  'sentences': ['Image by  Allison  Calabrese /  CC BY 4.0  of gluc

In [21]:
df = pd.DataFrame(pages_and_texts)

In [22]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1
1,-40,0,1,1,0.00,,[],0
2,-39,320,54,11,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1
3,-38,212,32,2,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1
4,-37,797,145,1,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2


In [23]:
df.tail()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
1203,1162,1676,252,1,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...,"[39., Exercise 10.2 & 11.3 reused “Egg Oval Fo...",18
1204,1163,1617,254,6,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...,[Images / Pixabay License; “Pumpkin Cartoon Or...,10
1205,1164,1715,261,8,428.75,Flashcard Images Note: Most images in the fla...,[Flashcard Images Note: Most images in the fl...,13
1206,1165,1733,268,4,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...,"[ShareAlike 11., Organs reused “Pancreas Orga...",13
1207,1166,257,44,1,64.25,23. Vitamin D reused “The Functions of Vitamin...,"[23., Vitamin D reused “The Functions of Vitam...",3


In [24]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,11.05,287.00,10.32
std,348.86,560.38,95.76,8.95,140.10,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.00,134.00,5.00,190.50,5.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00
75%,864.25,1603.50,271.00,15.00,400.88,15.00
max,1166.00,2308.00,429.00,106.00,577.00,28.00


In [26]:
chunk_size = 10

In [27]:
def split_list(input_list: list[str], 
               slice_size: int=chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

In [28]:
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"], slice_size=chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [29]:
random.sample(pages_and_texts,k=1)

[{'page_number': 348,
  'page_char_count': 1754,
  'page_word_count': 314,
  'page_sentence_count_raw': 9,
  'page_token_count': 438.5,
  'text': 'cells will start to die.  • Congestive heart failure. Sometimes referred to as heart  failure, this condition indicates that the heart is not pumping  blood as well as it should. The heart is still working but it is not  meeting the body’s demand for blood and oxygen. If left  unchecked, it can progress to further levels of malfunction.  • Arrhythmia. This is an abnormal rhythm of the heart. The heart  may beat above one hundred beats per minute (known as  tachycardia) or below sixty beats per minute (known as  bradycardia), or the beats are not regular. The heart may not  be able to pump enough volume of blood to meet the body’s  needs.  • Heart valve problems. Stenosis is a condition wherein the heart  valves become compromised in their ability to open wide  enough to allow proper blood flow. When the heart valves do  not close tightly and

In [30]:
df = pd.DataFrame(pages_and_texts)

In [31]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy,sentence_chunks,num_chunks
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1,[[Human Nutrition: 2020 Edition]],1
1,-40,0,1,1,0.00,,[],0,[],0
2,-39,320,54,11,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1,[[Human Nutrition: 2020 Edition UNIVERSITY O...,1
3,-38,212,32,2,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1,[[Human Nutrition: 2020 Edition by University ...,1
4,-37,797,145,1,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2,[[Contents Preface University of Hawai‘i at ...,1


In [32]:
df.tail()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy,sentence_chunks,num_chunks
1203,1162,1676,252,1,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...,"[39., Exercise 10.2 & 11.3 reused “Egg Oval Fo...",18,"[[39., Exercise 10.2 & 11.3 reused “Egg Oval F...",2
1204,1163,1617,254,6,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...,[Images / Pixabay License; “Pumpkin Cartoon Or...,10,[[Images / Pixabay License; “Pumpkin Cartoon O...,1
1205,1164,1715,261,8,428.75,Flashcard Images Note: Most images in the fla...,[Flashcard Images Note: Most images in the fl...,13,[[Flashcard Images Note: Most images in the f...,2
1206,1165,1733,268,4,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...,"[ShareAlike 11., Organs reused “Pancreas Orga...",13,"[[ShareAlike 11., Organs reused “Pancreas Org...",2
1207,1166,257,44,1,64.25,23. Vitamin D reused “The Functions of Vitamin...,"[23., Vitamin D reused “The Functions of Vitam...",3,"[[23., Vitamin D reused “The Functions of Vita...",1


In [33]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,11.05,287.00,10.32,1.53
std,348.86,560.38,95.76,8.95,140.10,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.00,134.00,5.00,190.50,5.00,1.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00,1.00
75%,864.25,1603.50,271.00,15.00,400.88,15.00,2.00
max,1166.00,2308.00,429.00,106.00,577.00,28.00,3.00


In [36]:
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        joined_sentence_chunk = "".join(sentence_chunk).replace("  "," ").strip()
        joined_sentence_chunk = re.sub(r'\.(A-Z)', r'. \1', joined_sentence_chunk) 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        
        pages_and_chunks.append(chunk_dict)
    
len(pages_and_chunks)       

  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [37]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 522,
  'sentence_chunk': 'Insufficient intake of dietary vitamin A over time can also cause complete vision loss.In fact, vitamin A deficiency is the number one cause of preventable blindness worldwide.Vitamin A not only supports the vision function of eyes but also maintains the coverings and linings of the eyes.Vitamin A deficiency can lead to the dysfunction of the linings and coverings of the eye (eg.bitot spots), causing dryness of the eyes, a condition called 522 | Fat-Soluble Vitamins',
  'chunk_char_count': 470,
  'chunk_word_count': 75,
  'chunk_token_count': 117.5}]

In [38]:
df = pd.DataFrame(pages_and_chunks)

In [39]:
df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,-41,Human Nutrition: 2020 Edition,29,4,7.25
1,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00
2,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50
3,-37,Contents Preface University of Hawai‘i at Māno...,765,113,191.25
4,-36,Lifestyles and Nutrition University of Hawai‘i...,940,141,235.00


In [40]:
df.tail()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
1838,1164,Flashcard Images Note: Most images in the flas...,1298,169,324.50
1839,1164,Hazard Analysis Critical Control Points reused...,373,49,93.25
1840,1165,ShareAlike 11.Organs reused “Pancreas Organ An...,1277,164,319.25
1841,1165,Sucrose reused “Figure 03 02 05” by OpenStax B...,408,57,102.00
1842,1166,23.Vitamin D reused “The Functions of Vitamin ...,248,35,62.00


In [41]:
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,731.11,109.00,182.78
std,347.79,445.65,69.34,111.41
min,-41.00,12.00,3.00,3.00
25%,280.50,313.50,43.00,78.38
50%,586.00,745.00,111.00,186.25
75%,890.00,1112.00,168.00,278.00
max,1166.00,1824.00,290.00,456.00


In [42]:
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count : {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count : 17.75 | Text: Table 6.1 Essential and Nonessential Amino Acids Defining Protein | 365
Chunk token count : 25.75 | Text: Snowdon W, Osborn T. (2003).Coconut: It’s role in health.Secretariat of the Pacific. 292 | Introduction
Chunk token count : 11.5 | Text: PART V CHAPTER 5.LIPIDS Chapter 5.Lipids | 289
Chunk token count : 28.5 | Text: The World Bank claims that a million deaths occur every year from anemia and that the majority of those Iron | 663
Chunk token count : 15.25 | Text: PART XVII CHAPTER 17.FOOD SAFETY Chapter 17.Food Safety | 985


In [43]:
#filtering rows with token under 30
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [44]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': 581,
  'sentence_chunk': 'Age Group RDA Males and Females mcg DFE/day UL Infants (0–6 months) 65* Not possible to determine Infants (7–12 months) 80* Not possible to determine Children (1–3 years) 150 300 Children (4–8 years) 200 400 Children (9–13 years) 300 600 Adolescents (14–18 years) 400 800 Adults (> 19 years) 400 1000 *denotes Adequate Intake Dietary Supplement Fact Sheet: Folate.National Institute of Health, Office of Dietary Supplements.https://ods.od.nih.gov/factsheets/ Folate-HealthProfessional/. Updated April 20, 2016.Accessed October 22, 2017. Dietary Sources Folate is found naturally in a wide variety of food especially in dark leafy vegetables, fruits, and animal products.The U.S. Food and Drug Administration (FDA) began requiring manufacturers to fortify enriched breads, cereals, flours, and cornmeal to increase the consumption of folate in the American diet.For the folate content of various foods, see Table 9.26 “Folate Content of Various Foods”. Table

## Text Embedding

In [65]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.

In [45]:
from sentence_transformers import SentenceTransformer
import torch

2025-08-02 12:01:58.609087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754136118.788969      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754136118.839044      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer("all-mpnet-base-v2", device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [48]:
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/1680 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
pages_and_chunks_over_min_token_len[0]["embedding"].shape

(768,)

In [50]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
text_chunks[9]

'Defining Protein University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 363 The Role of Proteins in Foods: Cooking and Denaturation University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 374 Protein Digestion and Absorption University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 378 Protein’s Functions in the Body University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 383 Diseases Involving Proteins University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 395 Proteins in a Nutshell University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 405 Proteins, Diet, and Personal Choices University of Hawai‘i at Mānoa Food Science and Human Nutrition Program and Human Nutrition Program 409'

In [51]:
len(text_chunks)

1680

In [52]:
text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=16, convert_to_tensor=True)
text_chunk_embeddings[0]

Batches:   0%|          | 0/105 [00:00<?, ?it/s]

tensor([ 6.7424e-02,  9.0228e-02, -5.0955e-03, -3.1755e-02,  7.3908e-02,
         3.5198e-02, -1.9799e-02,  4.6769e-02,  5.3573e-02,  5.0123e-03,
         3.3393e-02, -1.6222e-03,  1.7608e-02,  3.6265e-02, -3.1664e-04,
        -1.0712e-02,  1.5426e-02,  2.6218e-02,  2.7766e-03,  3.6494e-02,
        -4.4411e-02,  1.8936e-02,  4.9012e-02,  1.6402e-02, -4.8578e-02,
         3.1829e-03,  2.7299e-02, -2.0475e-03, -1.2283e-02, -7.2805e-02,
         1.2045e-02,  1.0730e-02,  2.1000e-03, -8.1777e-02,  2.6783e-06,
        -1.8143e-02, -1.2080e-02,  2.4718e-02, -6.2747e-02,  7.3544e-02,
         2.2162e-02, -3.2877e-02, -1.8010e-02,  2.2295e-02,  5.6136e-02,
         1.7951e-03,  5.2593e-02, -3.3174e-03, -8.3388e-03, -1.0628e-02,
         2.3192e-03, -2.2393e-02, -1.5301e-02, -9.9306e-03,  4.6532e-02,
         3.5747e-02, -2.5476e-02,  2.6369e-02,  3.7491e-03, -3.8268e-02,
         2.5833e-02,  4.1287e-02,  2.5818e-02,  3.3297e-02, -2.5178e-02,
         4.5152e-02,  4.4904e-04, -9.9662e-02,  4.9

## Saving Embeddings

In [53]:
#Saving embedding to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
save_path = "text-embeddings.csv"
text_chunks_and_embeddings_df.to_csv(save_path, index=False)

In [60]:
# Import saved file and view
text_chunks_and_embeddings_df = pd.read_csv("/kaggle/working/text-embeddings.csv")
text_chunks_and_embeddings_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242601e-02 9.02281925e-02 -5.09549258e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156307e-02 5.92139587e-02 -1.66167375e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,765,113,191.25,[ 2.79802028e-02 3.39813605e-02 -2.06426457e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,940,141,235.00,[ 6.82566911e-02 3.81274968e-02 -8.46853573e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264531e-02 -8.49764794e-03 9.57159977e-...


## Retrieval System

In [67]:
import numpy as np
import ast
from sentence_transformers import util

In [65]:
def parse_embedding_string(s):
    return np.fromstring(s.strip('[]'), sep=' ')

In [63]:
text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(parse_embedding_string)
embeddings = torch.tensor(
    np.stack(text_chunks_and_embeddings_df["embedding"].tolist(), axis=0),
    dtype=torch.float32
).to(device)

In [66]:
embeddings.shape

torch.Size([1680, 768])

In [69]:
query = "macronutrients functions"
query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")
dot_scores = util.dot_score(query_embedding, embeddings)[0]
top_results = torch.topk(dot_scores, k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
def retrieve_relevant_resources(query: str, n_resources_to_return: int=5):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    return scores, indices

In [72]:
retrieve_relevant_resources(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(tensor([0.6926, 0.6738, 0.6646, 0.6536, 0.6473], device='cuda:0'),
 tensor([42, 47, 41, 51, 46], device='cuda:0'))

In [146]:
def print_top_results_and_scores(query: str, n_resources_to_return: int=2):
    scores, indices = retrieve_relevant_resources(query, n_resources_to_return=n_resources_to_return)
    for score, idx in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print("Text")
        print(pages_and_chunks[idx]["sentence_chunk"])
        print("\n\n")

In [147]:
print_top_results_and_scores(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 0.6463
Text
People’s sensitivity to the adverse effects of caffeine varies and some people develop side effects at much lower doses.The many effects caffeine has on the brain do not diminish with habitual drinking of caffeinated beverages. Tools for Change Consuming caffeine in the evening and in the middle of the night will help keep you awake to study for an exam, but it will not enhance your performance on the next day’s test if you do not get enough sleep.Drink caffeinated beverages in moderation at any time of the day or evening to increase alertness (if you are not sensitive to caffeine’s adverse effects), but get the recommended amount of sleep. It is important to note that caffeine has some effects on health that are either promoted or masked by the other beneficial 218 | Popular Beverage Choices



Score: 0.6298
Text
Table 2.1 The Eleven Organ Systems in the Human Body and Their Major Functions Organ System Organ Components Major Function Cardiovascular heart, blood/lym

## Generative Model

In [76]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.5/876.5 kB 36.6 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [77]:
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [79]:
import keras
import keras_nlp

In [81]:
llm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en") 

Mounting files to /kaggle/input/gemma/keras/gemma_instruct_2b_en/3...


I0000 00:00:1754137649.003597      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14523 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [168]:
query = "How does saliva helps in digestion?"
outputs = llm.generate(query)
print(f"Answer without RAG:{outputs.replace(query, '')}")

Answer without RAG:

Saliva helps in digestion by:

- Breaking down carbohydrates into simple sugars
- Breaking down proteins into amino acids
- Breaking down fats into fatty acids and glycerol
- Lubricating the food bolus and facilitating its passage through the mouth and stomach
- Dissolving certain nutrients, such as calcium and iron
- Inhibiting the growth of harmful bacteria

Saliva also helps to maintain oral hygiene by removing bacteria and food particles from the mouth and preventing cavities and gum disease.


In [165]:
def prompt_formatter(query: str, context_items: list[dict], max_tokens: int = 256) -> str:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])
    base_prompt = """Based on the context next, please answer this query:\n
    {query}\n
    If context is not given to the answer, answer it according to your cognition
    context: {context}\n
    Give yourself room to think by extracting relevant passages from the context before answering the query.
    Don't return the thinking, only return the answer.
    Make sure your answers are as explanatory as possible.
    Use the following examples as reference for the ideal answer style.
    """ 
    prompt = base_prompt.format(context=context, query=query)
    words = prompt.split()
    if len(words) > max_tokens:
        words = words[:max_tokens]
    return ' '.join(words)

In [166]:
scores, indices = retrieve_relevant_resources(query=query)
context_items = [pages_and_chunks[i] for i in indices]
prompt = prompt_formatter(query=query, context_items=context_items)
print(prompt)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the context next, please answer this query: How does saliva helps in digestion? If context is not given to the answer, answer it according to your cognition context: - People’s sensitivity to the adverse effects of caffeine varies and some people develop side effects at much lower doses.The many effects caffeine has on the brain do not diminish with habitual drinking of caffeinated beverages. Tools for Change Consuming caffeine in the evening and in the middle of the night will help keep you awake to study for an exam, but it will not enhance your performance on the next day’s test if you do not get enough sleep.Drink caffeinated beverages in moderation at any time of the day or evening to increase alertness (if you are not sensitive to caffeine’s adverse effects), but get the recommended amount of sleep. It is important to note that caffeine has some effects on health that are either promoted or masked by the other beneficial 218 | Popular Beverage Choices - Table 2.1 The Ele

In [169]:
outputs = llm.generate(prompt, max_length=1024) 
print(f"Answer with RAG:\n{outputs.replace(prompt, '')}")

Answer with RAG:
, sensory perception, reproduction, excretion, and more.

Saliva helps to lubricate the mouth and throat, and it also helps to break down food particles. Saliva contains enzymes that help to break down carbohydrates, proteins, and fats. It also contains antibacterial agents that help to protect the body from infection.

Therefore, saliva helps to promote the digestion of food particles by lubricating the mouth and throat, breaking down food particles with enzymes, and protecting the body from infection.
